In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString
import geopandas as gpd


client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [22]:
num_rows_batch_publish=5000

In [35]:
magnitudes = ["co", "pm10", "pm2.5", "no2", "o3"] # to be used
gdf_property_list=["co", "pm10", "pm2_5", "no2",  "o3"]
entity_id_prefix =  "urn:ngsi-ld:AirQualityObserved:Berlin"
entity_type = 'AirQualityObserved'


In [48]:
pub_df = {}
gdf_no2 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_no2.pkl')
gdf_o3 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_o3.pkl')
gdf_pm10 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_pm10.pkl')
gdf_pm2_5 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_pm2.5.pkl')
gdf_co = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_co.pkl')

In [49]:
gdf_list= [gdf_co, gdf_pm10, gdf_pm2_5, gdf_no2,  gdf_o3 ]

In [50]:
gdf_co

,datetime,station_id,co,geometry,longitude,latitude,data,hexagon_id,hexagon_x,hexagon_y,lat,lon
0,01.01.2022 00:00,117 Schildhornstraße,0.2,POINT (1482578.185 6884388.306),13.318226,52.463600,0.2,49,1,4,52.468799,13.324601
1,01.01.2022 00:00,174 Frankfurter Allee,0.2,POINT (1499486.390 6893608.456),13.470115,52.514034,0.2,13,-2,-1,52.511793,13.463448
2,01.01.2022 01:00,174 Frankfurter Allee,0.3,POINT (1499486.390 6893608.456),13.470115,52.514034,0.3,13,-2,-1,52.511793,13.463448
3,01.01.2022 01:00,117 Schildhornstraße,0.3,POINT (1482578.185 6884388.306),13.318226,52.463600,0.3,49,1,4,52.468799,13.324601
4,01.01.2022 02:00,174 Frankfurter Allee,0.4,POINT (1499486.390 6893608.456),13.470115,52.514034,0.4,13,-2,-1,52.511793,13.463448
...,...,...,...,...,...,...,...,...,...,...,...,...
17513,31.12.2022 21:00,174 Frankfurter Allee,0.2,POINT (1499486.390 6893608.456),13.470115,52.514034,0.2,13,-2,-1,52.511793,13.463448
17514,31.12.2022 22:00,117 Schildhornstraße,0.1,POINT (1482578.185 6884388.306),13.318226,52.463600,0.1,49,1,4,52.468799,13.324601
17515,31.12.2022 22:00,174 Frankfurter Allee,0.2,POINT (1499486.390 6893608.456),13.470115,52.514034,0.2,13,-2,-1,52.511793,13.463448
17516,31.12.2022 23:00,117 Schildhornstraße,0.2,POINT (1482578.185 6884388.306),13.318226,52.463600,0.2,49,1,4,52.468799,13.324601


In [30]:
gdf_pm2_5

,datetime,station_id,pm2.5,geometry,longitude,latitude,hexagon_id,hexagon_x,hexagon_y,lat,lon
0,01.01.2022 00:00,077 Buch,7,POINT (1500730.879 6917185.795),13.481295,52.642739,147,1,-6,52.651207,13.488535
1,01.01.2022 00:00,032 Grunewald,10,POINT (1472195.462 6886120.669),13.224957,52.473080,111,4,6,52.480502,13.234906
2,01.01.2022 00:00,190 Leipziger Straße,12,POINT (1489719.398 6892835.849),13.382377,52.509810,2,0,1,52.507496,13.392682
3,01.01.2022 00:00,010 Wedding,19,POINT (1486043.152 6898882.117),13.349353,52.542855,10,2,1,52.539674,13.354664
4,01.01.2022 00:00,117 Schildhornstraße,17,POINT (1482578.185 6884388.306),13.318226,52.463600,49,1,4,52.468799,13.324601
...,...,...,...,...,...,...,...,...,...,...,...
105103,31.12.2022 23:00,077 Buch,10,POINT (1500730.879 6917185.795),13.481295,52.642739,147,1,-6,52.651207,13.488535
105104,31.12.2022 23:00,221 Karl-Marx-Straße,18,POINT (1495891.174 6887185.481),13.437819,52.478907,20,-2,1,52.475286,13.430667
105105,31.12.2022 23:00,010 Wedding,13,POINT (1486043.152 6898882.117),13.349353,52.542855,10,2,1,52.539674,13.354664
105106,31.12.2022 23:00,171 Mitte,28,POINT (1493774.044 6893531.107),13.418801,52.513611,5,-1,0,52.509650,13.428062


In [52]:

example_flag=False
for property_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    prop =gdf_property_list[property_index]

    hexagon_id_list = tmp_gdf['hexagon_id'].unique()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]
        
        entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity

        counter =0
        for index, row in hexagon_gdf.iterrows():
            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lat, lon))
                    entity_obj.add_geoprop("location", location_value, observedAt= datetime.strptime(str(row.datetime), '%d.%m.%Y %H:%M')) # MAKE THIS A POINT (SIVA EXAMPLE)
                elif feature_list[f] in ["co", "pm10", "pm2.5", "no2", "o3"]:
                    if(feature_list[f] == "pm2.5"):
                        print(feature_list[f])
                        entity_obj.add_prop("pm2_5", str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%d.%m.%Y %H:%M'))                 
                    else :
                        entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%d.%m.%Y %H:%M'))
                #elif(feature_list[f] not in ['datetime','type']):
                #    entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully' 
        
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


Info: Unique hexagons for the dataset : [49 13]
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:AirQualityObserved:Berlin:Hexagon:49', 'type': 'AirQualityObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'co': {'type': 'Property', 'value': '0.2', 'observedAt': '2022-01-01T00:00:00Z'}, 'location': {'type': 'GeoProperty', 'value': {"coordinates": [52.468799, 13.324601], "type": "Point"}, 'observedAt': '2022-01-01T00:00:00Z'}}
... published data for hexagon: 49
... published data for hexagon: 13
Info: Dataset is published successfully --- It took 5.83543018065393 seconds to publish the dataset with length 17518
Info: Unique hexagons for the dataset : [147 111   2  10  49 236  41  20  14  13  61   5]
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:AirQualityObserved:Berlin:Hexagon:147', 'type': 'AirQualityObserved', '@context': ['https: